### IMPORTS

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from itertools import chain

# Standard library
from pathlib import Path
from typing import TYPE_CHECKING

import networkx as nx

# Third-party libraries
import numpy as np
from rich.console import Console

# Local libraries

if TYPE_CHECKING:
    from collections.abc import Generator

# Global constants
# SCRIPT_NAME = __file__.split("/")[-1][:-3]
SCRIPT_NAME = "canical_test"
CWD = Path.cwd()
DATA = Path(CWD / "__data__" / SCRIPT_NAME)
DATA.mkdir(exist_ok=True, parents=True)
SEED = 42

# Global functions
console = Console()
console.is_jupyter = False
RNG = np.random.default_rng(SEED)


from builtins import type as create_class
from collections.abc import Sequence
from dataclasses import dataclass
from enum import Enum
from typing import Any, cast

import numpy as np
import quaternion
from numpy.typing import NDArray

from ariel.body_phenotypes.robogen_lite.config import (
    ALLOWED_FACES,
    ALLOWED_ROTATIONS,
    ModuleFaces,
    ModuleInstance,
    ModuleRotationsIdx,
    ModuleType,
)

# * IT IS SUPER VERY IMPORTANT THAT MODULEFACES ORDER NEVER CHANGES<3

---

### HELPER CLASS FOR ANGLES

In [ ]:
@dataclass
class QuatHolder:
    float_array: np.ndarray | Sequence[float | int]

    _SHIFT: int = -1
    _DECIMALS: int = 3
    _ROT_ANGLE = 45
    _SINGLE_ROTATION_STEP: tuple[int, ...] = (180, -(180 - _ROT_ANGLE), 0)
    _MUJOCO_NORMAL_VECTOR: tuple[int, ...] = (0, 1, 0)

    def __post_init__(self) -> None:
        if isinstance(self.float_array, np.ndarray):
            return

        self.float_array = np.roll(
            qnp.as_float_array(
                qnp.from_euler_angles(np.deg2rad(self.float_array))
            ),
            shift=self._SHIFT,
        )

    @property
    def quat(self) -> NDArray[np.quaternion]:
        return qnp.from_float_array(self.float_array)

    @property
    def unit_vectors(self) -> np.ndarray:  # Shape (3, 3)
        quat = self.quat
        local_vectors = np.eye(3)  # Rows: X, Y, Z
        rotated = np.array([
            qnp.rotate_vectors(quat, vec) for vec in local_vectors
        ])
        return np.round(rotated, decimals=self._DECIMALS)

    @property
    def normal_vector(self) -> np.ndarray:
        return np.array(self._MUJOCO_NORMAL_VECTOR) @ self.unit_vectors

    def rotate(
        self, angle: int | None = None, axis: tuple[int, ...] | None = None
    ) -> "QuatHolder":
        if angle is not None or axis is not None:
            angle_to_use = angle if angle is not None else self._ROT_ANGLE
            axis_to_use = (
                axis if axis is not None else self._MUJOCO_NORMAL_VECTOR
            )
            angle_rad = np.deg2rad(angle_to_use)
            axis_normalized = np.array(axis_to_use, dtype=float)
            axis_normalized /= np.linalg.norm(axis_normalized)
            rotation_vector = axis_normalized * angle_rad
            body_rotation_quat = qnp.from_rotation_vector(rotation_vector)
        else:
            body_rotation_quat = QuatHolder(self._SINGLE_ROTATION_STEP).quat

        rotated = body_rotation_quat * self.quat
        return QuatHolder(qnp.as_float_array(rotated))

    def __repr__(self) -> str:
        return f"QuatHolder({np.round(self.float_array, self._DECIMALS)})"

    def __hash__(self) -> int:
        quat_tuple = (
            tuple(self.float_array.flat)
            if isinstance(self.float_array, np.ndarray)
            else tuple(self.quat)
        )
        return hash((quat_tuple, self._SHIFT, self._DECIMALS))

    def __eq__(self, other) -> bool:
        q1 = self.quat
        q2 = other.quat
        tolerance = 10 ** (-self._DECIMALS)
        return np.allclose(q1, q2, atol=tolerance) or np.allclose(
            q1, -q2, atol=tolerance
        )

---

#### FILL IN THE CONFIG WITH GIVEN ANGLES >WIP<

In [ ]:
class SymmetryPlane(Enum):
    """Symmetry plane determines how many unique rotations a module has."""

    NONE = 1
    TWO_FOLD = 2
    FOUR_FOLD = 4


class Priority(Enum):
    FIRST = 3
    SECOND = 2
    THIRD = 1
    LAST = 0


FACE_QUATS: dict[ModuleFaces, QuatHolder] = {
    ModuleFaces.BACK: QuatHolder((0, 0, 0)),
    ModuleFaces.FRONT: QuatHolder((0, 180, 180)),
    ModuleFaces.RIGHT: QuatHolder((90, -90, -90)),
    ModuleFaces.LEFT: QuatHolder((90, 90, -90)),
    ModuleFaces.TOP: QuatHolder((0, 0, -90)),
    ModuleFaces.BOTTOM: QuatHolder((0, 180, 90)),
}


@dataclass(frozen=True)
class ModuleCanonicalConfig:
    """Pre-computed canonical configuration for a module type."""

    axial_face_order: tuple[ModuleFaces, ...]
    radial_face_order: tuple[ModuleFaces, ...]
    radial_shift: int

    symmetry_plane: SymmetryPlane

    max_allowed_rotations: int
    priority: Priority
    radial_adjustments: dict[int, tuple[int, ...]]

    @classmethod
    def create(
        cls,
        module_type: ModuleType,
        axial_faces: set[ModuleFaces] | None,
        radial_faces: set[ModuleFaces] | None,
        attachment_face: ModuleFaces | None,
        face_quats: dict[ModuleFaces, QuatHolder] | None,
        symmetry_plane: SymmetryPlane,
        allowed_rotations: list[ModuleRotationsIdx],
        priority: Priority,
        # attachment_face: ModuleFaces | None,
    ) -> "ModuleCanonicalConfig":
        """Factory method to create a config with computed face orders."""
        # axial_face_order, radial_face_order = cls._compute_face_indices(
        #     module_type, attachment_face
        # )
        max_allowed_rotations = cls._compute_max_allowed_rotations(
            allowed_rotations, symmetry_plane
        )

        axial_face_order, radial_face_order, radial_shift, rotation_axis = (
            cls._calculate_face_order(
                axial_faces,
                radial_faces,
                face_quats,
                attachment_face,
                max_allowed_rotations,
            )
        )

        radial_adjustments = cls._calculate_face_angle_adjustments(
            radial_face_order,
            radial_shift,
            rotation_axis,
            max_allowed_rotations,
            face_quats,
        )

        return cls(
            axial_face_order=axial_face_order,
            radial_face_order=radial_face_order,
            radial_shift=radial_shift,
            symmetry_plane=symmetry_plane,
            max_allowed_rotations=max_allowed_rotations,
            priority=priority,
            radial_adjustments=radial_adjustments,
        )

    # todo: big mess. also need early returns
    @staticmethod
    def _calculate_face_order(
        axial_faces: set[ModuleFaces],
        radial_faces: set[ModuleFaces],
        face_quats: dict[ModuleFaces, QuatHolder],
        attachment_face: ModuleFaces | None,
        max_allowed_rotations: int,
    ) -> tuple[list[ModuleFaces | None], list[ModuleFaces | None], int]:
        """
        start oder biased and based on positive or negative direction normal vector
        """
        rotation_axis = None
        list_axial_faces = list(axial_faces)

        axial_faces_order = []
        if len(axial_faces) == 2:
            for axis in range(3):
                if (
                    face_quats[list_axial_faces[0]].normal_vector[axis]
                    + face_quats[list_axial_faces[1]].normal_vector[axis]
                    == 0
                    and face_quats[list_axial_faces[1]].normal_vector[axis] != 0
                ):
                    rotation_axis = axis

                    if face_quats[list_axial_faces[0]].normal_vector[axis] > 0:
                        axial_faces_order.append(list_axial_faces[0])
                        axial_faces_order.append(list_axial_faces[1])
                    else:
                        axial_faces_order.append(list_axial_faces[1])
                        axial_faces_order.append(list_axial_faces[0])

                    if attachment_face and attachment_face in axial_faces_order:
                        axial_faces_order.remove(attachment_face)

        def sort_key(face: ModuleFaces) -> tuple[int, int]:
            normal = face_quats[face].normal_vector
            axis = np.argmax(np.abs(normal))
            sign = (
                0 if normal[axis] > 0 else 1
            )  # 0 = positive (first), 1 = negative (second)
            return (sign, axis)

        def find_shift_validated(
            original: list[np.ndarray], rotated: list[np.ndarray]
        ) -> int:
            """Find shift and validate all elements align."""
            n = len(original)
            for shift in range(n):
                if all(
                    np.allclose(rotated[i], original[(i + shift) % n])
                    for i in range(n)
                ):
                    return shift

            raise ValueError(
                "No valid shift found - lists don't match under any rotation!"
            )

        radial_face_order = sorted(radial_faces, key=sort_key)
        # print(radial_face_order)

        axis = [0, 0, 0]
        axis[rotation_axis] = 1
        # determine the rotation direction:

        # normal_vectors ordered
        radial_face_quats = [face_quats[face] for face in radial_face_order]
        # print(radial_face_quats)
        normal_face_quats = [
            face_quats[face].normal_vector for face in radial_face_order
        ]
        # print(normal_face_quats)

        rotated_radial_face_quatas = radial_face_quats.copy()
        # print("max allowd rotations",max_allowed_rotations)
        for _ in range(max_allowed_rotations):
            # print('rotating once', rotated_radial_face_quatas)
            # print(axis)
            rotated_radial_face_quatas = [
                quat.rotate(axis=tuple(axis))
                for quat in rotated_radial_face_quatas
            ]
        # print(rotated_radial_face_quatas)

        # print('normal vectors after a rotation:')

        rotated_normal_face_quats = [
            quat.normal_vector for quat in rotated_radial_face_quatas
        ]
        # print(rotated_normal_face_quats)

        radial_shift = find_shift_validated(
            normal_face_quats, rotated_normal_face_quats
        )

        # shift = next(i for i, v in enumerate(normal_face_quats) if np.allclose(rotated_normal_face_quats[0], v))

        # todo: need to figure out what thiis shift means. do i need this to be negative or positive to return
        # print(shift)

        return axial_faces_order, radial_face_order, radial_shift, rotation_axis

    @staticmethod
    def _compute_max_allowed_rotations(
        allowed_rotations: list[ModuleRotationsIdx],
        symmetry_plane: SymmetryPlane,
    ) -> int:
        """Calculate max allowed rotations based on symmetry."""
        result = len(allowed_rotations) // symmetry_plane.value
        return result if result != 0 else symmetry_plane.value

    # TODO: finish!!!
    @staticmethod
    def _calculate_face_angle_adjustments(
        radial_face_order,
        radial_shift,
        rotation_axis,
        max_allowed_rotations,
        face_quats,
    ):
        if not radial_face_order:
            return {}

        n = len(radial_face_order)

        original_quats = [face_quats[face] for face in radial_face_order]

        print(original_quats)

        axis = [0, 0, 0]
        axis[rotation_axis] = 1
        print(axis)
        rotated_radial_face_quatas = original_quats.copy()

        shifted_quats = (
            rotated_radial_face_quatas[-radial_shift:]
            + rotated_radial_face_quatas[:-radial_shift]
        )
        print(shifted_quats)

        for _ in range(max_allowed_rotations):
            shifted_quats = [
                quat.rotate(axis=tuple(axis)) for quat in shifted_quats
            ]

        print(shifted_quats)

        adjustments = {}

        for i in range(len(original_quats)):
            amt_rotate = 0

            MAX = 8
            for _ in range(MAX):
                shifted_quats[i].rotate(
                    axis=tuple(shifted_quats[i].normal_vector)
                )

                if original_quats[i] == shifted_quats[i]:
                    print("hi")
                    adjustments[i] = amt_rotate
                    continue

    # todo: AI SLOP FOR DEBUGGING
    @staticmethod
    def _calculate_face_angle_adjustments(
        radial_face_order,
        radial_shift,
        rotation_axis,
        max_allowed_rotations,
        face_quats,
    ):
        """Calculate rotation adjustments for each radial position."""

        if not radial_face_order:
            print("⚠️  No radial faces - returning empty adjustments")
            return {}

        n = len(radial_face_order)
        print(f"\n{'=' * 60}")
        print(f"CALCULATING ADJUSTMENTS")
        print(f"{'=' * 60}")
        print(f"📋 Radial face order: {[f.name for f in radial_face_order]}")
        print(f"🔄 Radial shift: {radial_shift}")
        print(f"📐 Rotation axis: {rotation_axis} ({'XYZ'[rotation_axis]})")
        print(f"🔢 Max allowed rotations: {max_allowed_rotations}")

        # Get original quaternions
        original_quats = [face_quats[face] for face in radial_face_order]
        print(f"\n📦 Original quats:")
        for i, (face, quat) in enumerate(
            zip(radial_face_order, original_quats)
        ):
            print(f"  [{i}] {face.name:>6}: {quat}")

        # Create rotation axis vector
        axis = [0, 0, 0]
        axis[rotation_axis] = 1
        print(f"\n🎯 Rotation axis vector: {axis}")

        # Apply shift
        radial_shift = radial_shift
        shifted_quats = (
            original_quats[radial_shift:] + original_quats[:radial_shift]
        )
        print(f"\n🔀 After shift by {radial_shift}:")
        for i, quat in enumerate(shifted_quats):
            print(f"  [{i}] {quat}")

            print(f"  [{i}] {quat.unit_vectors}")

        # Rotate max_allowed_rotations times
        print(
            f"\n🔄 Rotating {max_allowed_rotations} times around axis {axis}..."
        )
        for rot_step in range(max_allowed_rotations):
            axis = quat.normal_vector
            shifted_quats = [
                quat.rotate(axis=tuple(axis)) for quat in shifted_quats
            ]
            print(f"  Step {rot_step + 1}:")
            for i, quat in enumerate(shifted_quats):
                print(f"    [{i}] {quat}")

                print(f"  [{i}] {quat.unit_vectors}")

        print(f"\n📊 Finding adjustments for each position:")
        adjustments = {}
        MAX = 8

        for i in range(n):
            print(f"\n  Position [{i}]:")
            print(f"    🎯 Target (original): {original_quats[i]}")

            print(f"    🎯 Target (Vectors): {original_quats[i].unit_vectors}")

            print(f"    📍 Current (shifted): {shifted_quats[i]}")

            print(f"    📍 Current (Vectors): {shifted_quats[i].unit_vectors}")

            test_quat = shifted_quats[i]
            amt_rotate = 0
            found = False

            # Check if already equal
            if original_quats[i] == test_quat:
                print(f"    ✅ Already equal! Adjustment: 0")
                adjustments[i] = (0,)
                found = True
            else:
                # Try rotating to match
                for step in range(1, MAX + 1):
                    test_quat = test_quat.rotate(axis=tuple(axis))
                    print(f"    🔄 After rotation {step}: {test_quat}")

                    print(f"  {test_quat.unit_vectors}")

                    if original_quats[i] == test_quat:
                        amt_rotate = step
                        print(f"    ✅ Match found! Adjustment: {amt_rotate}")
                        adjustments[i] = (amt_rotate,)
                        found = True
                        break

                if not found:
                    print(f"    ❌ No match found after {MAX} rotations!")
                    adjustments[i] = (None,)

        print(f"\n{'=' * 60}")
        print(f"📋 FINAL ADJUSTMENTS:")
        for pos, adj in adjustments.items():
            print(f"  Position {pos}: {adj}")
        print(f"{'=' * 60}\n")

        return adjustments


CALCULATING ADJUSTMENTS
📋 Radial face order: ['LEFT', 'FRONT', 'RIGHT', 'BACK']
🔄 Radial shift: 1
📐 Rotation axis: 2 (Z)
🔢 Max allowed rotations: 2

📦 Original quats:
  [0]   LEFT: QuatHolder([-0.707  0.     0.     0.707])
  [1]  FRONT: QuatHolder([1. 0. 0. 0.])
  [2]  RIGHT: QuatHolder([ 0.707 -0.     0.     0.707])
  [3]   BACK: QuatHolder([-0.  0.  0.  1.])

🎯 Rotation axis vector: [0, 0, 1]

🔀 After shift by 1:
  [0] QuatHolder([1. 0. 0. 0.])
  [0] [[ 1.  0. -0.]
 [ 0.  1.  0.]
 [ 0. -0.  1.]]
  [1] QuatHolder([ 0.707 -0.     0.     0.707])
  [1] [[-0.  1. -0.]
 [-1. -0. -0.]
 [-0.  0.  1.]]
  [2] QuatHolder([-0.  0.  0.  1.])
  [2] [[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]
  [3] QuatHolder([-0.707  0.     0.     0.707])
  [3] [[-0. -1.  0.]
 [ 1. -0. -0.]
 [ 0.  0.  1.]]

🔄 Rotating 2 times around axis [0, 0, 1]...
  Step 1:
    [0] QuatHolder([0.924 0.383 0.    0.   ])
  [0] [[ 1.     0.    -0.   ]
 [ 0.     0.707  0.707]
 [ 0.    -0.707  0.707]]
    [1] QuatHolder([ 0.653  

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:539                                                                                  │
│                                                                                                  │
│   536 │   │   allowed_rotations=ALLOWED_ROTATIONS[ModuleType.BRICK],                             │
│   537 │   │   priority=Priority.SECOND,                                                          │
│   538 │   ),                                                                                     │
│ ❱ 539 │   ModuleType.HINGE: ModuleCanonicalConfig.create(                                        │
│   540 │   │   module_type=ModuleType.HINGE,                                                      │
│   541 │   │   axial_faces={ModuleFaces.FRONT},                                                   │
│   542 │   │   radial_faces=None,                                                                 │
│                                                                                                  │
│ in create:57                                                                                     │
│                                                                                                  │
│    54 │   │   │   allowed_rotations, symmetry_plane                                              │
│    55 │   │   )                                                                                  │
│    56 │   │                                                                                      │
│ ❱  57 │   │   axial_face_order, radial_face_order, radial_shift, rotation_axis = cls._calculat   │
│    58 │   │   │   axial_faces,                                                                   │
│    59 │   │   │   radial_faces,                                                                  │
│    60 │   │   │   face_quats,                                                                    │
│                                                                                                  │
│ in _calculate_face_order:139                                                                     │
│                                                                                                  │
│   136 │   │   │                                                                                  │
│   137 │   │   │   raise ValueError("No valid shift found - lists don't match under any rotatio   │
│   138 │   │                                                                                      │
│ ❱ 139 │   │   radial_face_order = sorted(radial_faces, key=sort_key)                             │
│   140 │   │   # print(radial_face_order)                                                         │
│   141 │   │                                                                                      │
│   142 │   │   axis = [0,0,0]                                                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: 'NoneType' object is not iterable

In [ ]:
"""
LEFT
[[-0. -1.  0.]
 [ 1. -0. -0.]
 [ 0.  0.  1.]]

FRONT
[[ 1.  0. -0.]
 [ 0.  1.  0.]
 [ 0. -0.  1.]]

RIGHT
[[-0.  1. -0.]
 [-1. -0. -0.]
 [-0.  0.  1.]]

BACK
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]



-----------------------------------
LEFT
[[-0. -1.  0.]
 [ 1. -0. -0.]
 [ 0.  0.  1.]]

BOTTOM
[[ 1.  0.  0.]
 [ 0. -0.  1.]
 [ 0. -1. -0.]]

RIGHT
[[ 0. -0.  1.]
 [-1. -0.  0.]
 [ 0. -1.  0.]]

TOP
[[-1.  0.  0.]
 [ 0. -0. -1.]
 [ 0. -1.  0.]]

"""


╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ """MUJOCO_LOG.TXT                                                                                │
│ ▲                                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: incomplete input

---

In [ ]:
# Pre-define configurations for each module type
CANONICAL_CONFIGS: dict[ModuleType, ModuleCanonicalConfig] = {
    ModuleType.CORE: ModuleCanonicalConfig.create(
        module_type=ModuleType.CORE,
        axial_faces={ModuleFaces.TOP, ModuleFaces.BOTTOM},
        radial_faces={
            ModuleFaces.LEFT,
            ModuleFaces.FRONT,
            ModuleFaces.RIGHT,
            ModuleFaces.BACK,
        },
        attachment_face=None,
        face_quats=FACE_QUATS,
        symmetry_plane=SymmetryPlane.FOUR_FOLD,
        allowed_rotations=ALLOWED_ROTATIONS[ModuleType.BRICK], #!  purposfully changed this.
        priority=Priority.FIRST,
    ),
    ModuleType.BRICK: ModuleCanonicalConfig.create(
        module_type=ModuleType.BRICK,
        axial_faces={ModuleFaces.FRONT, ModuleFaces.BACK},
        radial_faces={
            ModuleFaces.LEFT,
            ModuleFaces.TOP,
            ModuleFaces.RIGHT,
            ModuleFaces.BOTTOM,
        },
        attachment_face=ModuleFaces.BACK,
        face_quats=FACE_QUATS,
        symmetry_plane=SymmetryPlane.FOUR_FOLD,
        allowed_rotations=ALLOWED_ROTATIONS[ModuleType.BRICK],
        priority=Priority.SECOND,
    ),
    ModuleType.HINGE: ModuleCanonicalConfig.create(
        module_type=ModuleType.HINGE,
        axial_faces={ModuleFaces.FRONT},
        radial_faces=None,
        face_quats=FACE_QUATS,
        attachment_face=ModuleFaces.BACK,
        symmetry_plane=SymmetryPlane.TWO_FOLD,
        allowed_rotations=ALLOWED_ROTATIONS[ModuleType.HINGE],
        priority=Priority.THIRD,
    ),
    ModuleType.NONE: ModuleCanonicalConfig.create(
        module_type=ModuleType.NONE,
        axial_faces=None,
        radial_faces=None,
        face_quats=None,
        attachment_face=ModuleFaces.BACK,
        symmetry_plane=SymmetryPlane.NONE,
        allowed_rotations=ALLOWED_ROTATIONS[ModuleType.NONE],
        priority=Priority.LAST,
    ),
}

---

### CLASS

In [ ]:
from collections import deque
from typing import Any, Generator

from ariel.body_phenotypes.robogen_lite.config import ModuleFaces


# TODO: add new config stuff in here
# todo: like shift, and new adjustment thing
class CanonicalNode:
    _id_counter = 0

    def __init__(
        self,
        module_type: ModuleType | str,
        rotation: int = 0,
        *,
        parent: CanonicalNode | None = None,
        config: ModuleCanonicalConfig | None = None,
        node_id: int | None = None,
    ) -> None:
        if isinstance(module_type, str):
            module_type = ModuleType[module_type.upper()]

        if config is None:
            config = CANONICAL_CONFIGS[module_type]

        if node_id is None:
            node_id = CanonicalNode._id_counter
            CanonicalNode._id_counter += 1

        self.module_type = module_type
        self.rotation = rotation
        self.node_id = node_id
        self.parent = parent

        self._symmetry_plane = config.symmetry_plane.value
        self._max_allowed_rotatations = config.max_allowed_rotations
        self._priority = config.priority.value
        self._full_priority = config.priority.value

        # List where children get stored and the face type mapping
        self._axial_face_order = config.axial_face_order
        self._radial_face_order = config.radial_face_order

        self._axial_list: list[None | CanonicalNode] = [None] * len(
            config.axial_face_order
        )
        self._radial_list: list[None | CanonicalNode] = [None] * len(
            config.radial_face_order
        )

    # region helpers

    def _get_face_index(self, face_name: str) -> tuple[str, int] | None:
        try:
            face = ModuleFaces[face_name.upper()]
            if face in self._axial_face_order:
                return ("_axial_list", self._axial_face_order.index(face))
            if face in self._radial_face_order:
                return ("_radial_list", self._radial_face_order.index(face))

            raise ValueError(
                f"Face '{face.name}' not allowed for {self.module_type.name}.\n"
                f"Allowed axial: {[f.name for f in self._axial_face_order]}\n"
                f"Allowed radial: {[f.name for f in self._radial_face_order]}"
            )
        except KeyError:
            return None  # Not a face at all

    def _copy_subtree(self) -> "CanonicalNode":
        # Create a copy of this node with a new ID
        new_node = CanonicalNode(
            module_type=self.module_type,
            rotation=self.rotation,
            node_id=None,  # Will get a new auto-generated ID
        )
        new_node.parent = None  # Clear parent
        new_node._full_priority = self._full_priority  # Copy priority state

        # Recursively copy children
        for face, child in self.children:
            if child is not None:
                child_copy = child._copy_subtree()
                new_node[face] = child_copy

        return new_node

    # Region dunder methods
    def __getattr__(self, name: str) -> CanonicalNode | None:
        try:
            result = self._get_face_index(name)
            if result is not None:
                list_name, index = result
                return getattr(self, list_name)[index]
        except ValueError:
            raise  # Re-raise for non-allowed faces
        raise AttributeError(
            f"'{self.__class__.__name__}' has no attribute '{name}'"
        )

    def __setattr__(self, name: str, child_node) -> None:
        try:
            result = self._get_face_index(name)
        except ValueError as e:
            raise ValueError(str(e)) from None  # Suppress traceback chaining

        if result is not None:
            list_name, index = result
            if isinstance(child_node, CanonicalNode):
                if child_node.parent:
                    child_node = child_node._copy_subtree()

                child_node.parent = self
                node = self

                # # todo: set the euler as None if its front else Not None
                # if list_name == "_radial_list":
                #     print(self._radial_face_quat[index].value)
                #     print(self._rotation_quat())
                #     child_node.quat = self._radial_face_quat[index].value * self._rotation_quat()

                while node:
                    node._full_priority += child_node._priority
                    node = node.parent
            else:
                raise ValueError("Trying to add a non CanonicalNode object")

            getattr(self, list_name)[index] = child_node
        else:
            object.__setattr__(self, name, child_node)

    def __setitem__(self, face: ModuleFaces, child: CanonicalNode) -> None:
        setattr(self, face.name, child)

    def __getitem__(self, face: ModuleFaces) -> CanonicalNode | None:
        return getattr(self, face.name)

    # TODO: make use of the rich tree print thing
    def __str__(self, level: int = 0) -> str:
        """
        Return a formatted string representation of the CanonicalNode object.

        Parameters
        ----------
        level : int, default 0
            Indentation level for nested string formatting.

        Returns
        -------
        str
            Multi-line formatted string showing node attributes and children.

        Notes
        -----
        - Creates hierarchical indentation using 2 spaces per level
        - Shows _axial_list children only for faces in ALLOWED_FACES[self.type]
        - Shows _radial_list children for all faces in ALLOWED_FACES[self.type]
        - Displays 'None' for missing child nodes at each face position
        """
        indent = "    " * level

        # Start with basic attributes
        attributes = [
            f"{indent}id={self.node_id}",
            f"{indent}type={self.module_type.name}",
            f"{indent}rotation={self.rotation}",
            # f"{indent}quat={self.quat}"
            # f"{indent}symmetry_plane={self._symmetry_plane}",
        ]

        # Add axial children
        axial_children = ",\n".join(
            f"{indent}  {face.name}: {getattr(self, face.name).__str__(level + 1)}"
            if getattr(self, face.name) is not None
            else f"{indent}  {face.name}: None"
            for face in self._axial_face_order  # Instance-specific
        )
        attributes.append(
            f"{indent}axial_side=[\n{axial_children or f'{indent}  None'}\n{indent}]",
        )

        # Add radial children
        radial_children = ",\n".join(
            f"{indent}  {face.name}: {getattr(self, face.name).__str__(level + 1)}"
            if getattr(self, face.name) is not None
            else f"{indent}  {face.name}: None"
            for face in self._radial_face_order  # Instance-specific
        )
        attributes.append(
            f"{indent}radial_side=[\n{radial_children or f'{indent}  None'}\n{indent}]",
        )

        # Format the final string iteratively
        formatted_attributes = "\n".join(attributes)
        return f"{indent}CanonicalNode(\n{formatted_attributes}\n{indent})"

    def __repr__(self) -> str:
        """
        Return a string representation of the object.

        Returns
        -------
        str
            A compact string representation showing type, rotation, and children.

        Notes
        -----
        - Format: "{type_initial}{rotation}[{face_children}]"
        - Type initial is first character of self.type.name
        - Rotation only included if non-zero
        - Face children shown as "{face_initial}:{child_repr}"
        - Empty brackets omitted if no children exist
        """
        repr_str = f"{self.module_type.name[0]}"
        if self.rotation != 0:
            repr_str += f"{int(self.rotation)}"
        repr_str += "{"

        children_repr = []
        for face in self._axial_face_order + self._radial_face_order:
            child = getattr(self, face.name)
            if child:
                children_repr.append(f"{face.name.lower()[0]}:{child!r}")
        repr_str += "".join(children_repr)
        repr_str += "}"
        return repr_str

    def __iter__(
        self,
    ) -> Generator[tuple[ModuleFaces, CanonicalNode], None, None]:
        """Allows iteration over (face, child) pairs for all non-None children."""
        yield from self.children

    # endregion

    # region properties
    @property
    def children(
        self,
    ) -> Generator[tuple[ModuleFaces, CanonicalNode], Any, None]:
        """Yields (face, child) tuples for all non-None children."""
        for face, child in chain(
            zip(self._axial_face_order, self._axial_list),
            zip(self._radial_face_order, self._radial_list),
        ):
            if child is not None:
                yield (face, child)

    @property
    def axial_children(self) -> Generator[CanonicalNode, Any, None]:
        for child in self._axial_list:
            if child is not None:
                yield child

    @property
    def radial_children(self) -> Generator[CanonicalNode, Any, None]:
        for child in self._radial_list:
            if child is not None:
                yield child

    # endregion

    # region construction
    @classmethod
    def new_root(
        cls, module_type: ModuleType = ModuleType.CORE, **kwargs
    ) -> "CanonicalNode":
        """Create a new root with ID counter starting at 0."""
        cls._id_counter = 0
        return cls(module_type=module_type, **kwargs)

    @classmethod
    def from_graph(
        cls,
        graph: nx.DiGraph[Any],
        skip_type: ModuleType = ModuleType.NONE,
    ) -> CanonicalNode:
        """Build a CanonicalNode tree from a NetworkX graph."""

        # TODO: check typing
        def get_rotation_value(rotation_attr):
            return (
                ModuleRotationsIdx[rotation_attr].value
                if isinstance(rotation_attr, str)
                else rotation_attr
            )

        def create_node(
            node_id: int,
        ) -> CanonicalNode:
            attrs = graph.nodes[node_id]
            return CanonicalNode(
                module_type=ModuleType[attrs["type"]],
                rotation=get_rotation_value(attrs["rotation"]),
                node_id=node_id,
                # _parent_id=_parent.node_id,
            )

        def fill_in(_parent: CanonicalNode) -> None:
            for _, child_id, edge_data in graph.out_edges(
                _parent.node_id, data=True
            ):
                child_type = ModuleType[graph.nodes[child_id]["type"]]

                if child_type == skip_type:
                    continue

                child = create_node(child_id)
                _parent[ModuleFaces[edge_data["face"]]] = child
                fill_in(child)

        root_id = next(
            node for node in graph.nodes() if graph.in_degree(node) == 0
        )
        root = create_node(root_id)
        fill_in(root)

        return root

    @classmethod
    def from_dict(
        cls,
        data: dict[int, ModuleInstance],
        omit_none: bool = False,
    ) -> CanonicalNode:
        root = CanonicalNode.new_root()

        def fill_in(data: dict[int, ModuleType], parent: CanonicalNode) -> None:
            for face, node_id in data[parent.node_id].links.items():
                # if omit_none and data[node_id].type == ModuleType.NONE:
                #     continue
                child = CanonicalNode(
                    module_type=data[node_id].type,
                    # _parent_id=_parent.id,
                    rotation=data[node_id].rotation.value,
                    node_id=node_id,
                )
                parent[face] = child
                fill_in(data, child)

        fill_in(data, root)
        return root

    # endregion

    # region conversion

    def to_dict(self, *, omit_none: bool = False) -> dict[int, ModuleInstance]:
        result = {}

        def collect_node(node: CanonicalNode) -> None:
            if omit_none and node.module_type == ModuleType.NONE:
                return

            links = {}
            for face, child in node:
                if not omit_none or child.module_type != ModuleType.NONE:
                    links[face] = child.node_id

            result[node.node_id] = ModuleInstance(
                type=node.module_type,
                rotation=ModuleRotationsIdx(node.rotation),
                links=links,
            )

            for _, child in node.children:
                collect_node(child)

        collect_node(self)
        return result

    # TODO; is this cursed to have in the arguments?
    def to_graph(
        self, *, skip_type: ModuleType = ModuleType.NONE
    ) -> nx.DiGraph[Any]:
        """Generate a NetworkX graph from the CanonicalNode tree."""
        graph = nx.DiGraph()

        def traverse_node(node: CanonicalNode) -> None:
            graph.add_node(
                node.node_id,
                type=node.module_type.name,
                rotation=ModuleRotationsIdx(node.rotation).name,
            )

            for face, child in node.children:
                if child.module_type == skip_type:
                    print("found skyptype")  # Todo: check if this works well?
                    continue

                graph.add_node(
                    child.node_id,
                    type=child.module_type.name,
                    rotation=ModuleRotationsIdx(child.rotation).name,
                )
                graph.add_edge(
                    node.node_id,
                    child.node_id,
                    face=face.name,
                )
                traverse_node(child)

        traverse_node(self)
        return graph

    # endregion

    # TODO: need to fix this........ and think
    # def is_connected_to_axial_root(self) -> bool:
    #     if not hasattr(self, '_face_attached'):
    #         return False

    #     # Must be attached via FRONT to start the chain
    #     if self._face_attached != ModuleFaces.FRONT:
    #         return False

    #     console.print('_parent has face attached atribute ')
    #     # Check if _parent exists and is attached via TOP/BOTTOM, and grand_parent has _is_new_root
    #     if (self._parent and
    #         hasattr(self._parent, '_face_attached') and
    #         self._parent._face_attached in (ModuleFaces.TOP, ModuleFaces.BOTTOM) and
    #         self._parent._parent and
    #         self._parent._parent._is_new_root):
    #         return True

    #     # If not directly matching, recurse up the chain if _parent exists
    #     if self._parent:
    #         return self._parent.is_connected_to_axial_root()

    #     return False

    # TODO might add this if i want to account for symmetry in the hashing?..
    def invert_symmetry(self) -> None:
        pass

    def canonicalize_child_order(self) -> None:
        """
        uses the full priority to determine the child order
        always sets the max at first
        rotatation always in 1 direction
        """
        side_amt = len(self._radial_list)
        if side_amt == 0:
            return

        priority_list = []
        for child in self._radial_list:
            if child is None:
                priority_list.append(0)
            else:
                priority_list.append(child._full_priority)

        max_index = priority_list.index(max(priority_list))
        num_rotations = (side_amt - max_index) % side_amt
        for _ in range(num_rotations):
            self.rotate()

    def get_euler(self):
        pass
        # know the placement of where the child is attached to and give the idx?

    # def rotate(self):
    #     self.rotation = self.rotation - self._max_allowed_rotatations
    #     self._radial_list[:] = self._radial_list[-1:] + self._radial_list[:-1]

    #     # account for axial child rotation
    #     for child in self.axial_children:
    #         child.rotation += self._max_allowed_rotatations

    #     # ONLY radial children (and their descendants) need transformation
    #     for child in self.radial_children:
    #         # child._cascade_radial_transform()

    # def rotate(self):
    #     """Rotate this node and update all descendant reference frames."""
    #     rotation_steps = 1  # We're rotating by one step

    #     # Update this node's rotation value
    #     self.rotation = self.rotation - self._max_allowed_rotatations

    #     # Rotate the radial list itself
    #     old_radial_list = self._radial_list.copy()
    #     self._radial_list[:] = self._radial_list[-1:] + self._radial_list[:-1]

    #     # Compensate axial children (they inherit the rotation)
    #     for child in self.axial_children:
    #         child.rotation += self._max_allowed_rotatations
    #         # Axial children also need their radials transformed!
    #         if len(child._radial_list) > 0:
    #             # Axial children rotate WITH the parent, so shift = rotation amount
    #             child._transform_radial_subtree(rotation_steps * ROTATION_DIRECTION)

    #     # Transform radial children based on their NEW position
    #     for new_index, child in enumerate(self._radial_list):
    #         if child and len(child._radial_list) > 0:
    #             # Find where this child WAS before rotation
    #             old_index = old_radial_list.index(child)

    #             # Calculate the transformation
    #             shift = self._get_radial_transform_for_parent_rotation(
    #                 rotation_steps,
    #                 new_index
    #             )

    #             # Apply recursively to entire subtree
    #             child._transform_radial_subtree(shift)

    # def _transform_radial_subtree(self, shift: int) -> None:
    #     """
    #     Recursively transform radial lists of this node and ALL descendants.

    #     This maintains morphological equivalence when a parent rotates.
    #     """
    #     if len(self._radial_list) == 0:
    #         return

    #     # Shift this node's radial children
    #     self._radial_list[:] = (
    #         self._radial_list[shift:] + self._radial_list[:shift]
    #     )

    #     # Recursively apply to ALL children (both axial AND radial!)
    #     # This is critical: rotation affects the entire subtree
    #     for _, child in self.children:
    #         child._transform_radial_subtree(shift)

    def rotate(self):
        self.rotation = self.rotation - self._max_allowed_rotatations
        self._radial_list[:] = self._radial_list[-1:] + self._radial_list[:-1]

        # Compensate axial children rotations
        for child in self.axial_children:
            child.rotation += self._max_allowed_rotatations

        # Transform radial children AND their descendants
        for i, child in enumerate(self._radial_list):
            if child and len(child._radial_list) > 0:
                # Calculate shift based on which position this child moved TO
                shift = self._calculate_radial_shift(i)

                # Recursively apply shift to entire subtree
                child._apply_radial_shift_recursive(shift)

    def _apply_radial_shift_recursive(self, shift: int) -> None:
        """Recursively apply a radial shift to this node and all descendants."""
        if len(self._radial_list) == 0:
            return

        # Shift this node's radial children
        self._radial_list[:] = (
            self._radial_list[shift:] + self._radial_list[:shift]
        )

        # Recursively apply to all children (both axial and radial)
        for _, child in self.children:
            child._apply_radial_shift_recursive(shift)

    def _calculate_radial_shift(self, position_index: int) -> int:
        """
        Calculate how much to shift a child's radial list based on:
        - Its position in parent's radial list
        - Direction of parent rotation
        """
        # For 4-fold symmetry (4 radial faces)
        # When parent rotates by -90°, child at position i needs shift:
        # Position 0 (new RIGHT): shift +1
        # Position 1 (new FRONT): shift +1
        # Position 2 (new LEFT):  shift -1
        # Position 3 (new BACK):  shift -1

        if len(self._radial_list) == 4:
            return 1 if position_index < 2 else -1

        # Add logic for 2-fold symmetry if needed
        return 0

    # def _cascade_radial_shift(self) -> None:
    #     """Recursively shift radial lists of entire subtree."""
    #     for _, child in self.children:
    #         if len(child._radial_list) > 0:
    #             # Shift child's radial list by 1
    #             child._radial_list[:] = child._radial_list[-1:] + child._radial_list[:-1]

    #         # Recurse
    #         # Only recurse on RADIAL children (not axial!)
    #         for child in self.radial_children:
    #             child._cascade_radial_shift()

    # def _cascade_radial_transform(self) -> None:
    #     """
    #     Recursively apply the radial transformation to entire subtree.

    #     When a parent rotates around its axial axis, all radial children
    #     need their radial lists shifted by 2 positions to maintain morphology.
    #     """
    #     for _, child in self.children:
    #         if len(child._radial_list) > 0:
    #             # Shift by 2 positions (equivalent to 180° rotation in child's frame)
    #             child._radial_list[:] = (
    #                 child._radial_list[2:] + child._radial_list[:2]
    #             )

    #         # Recursively apply to all grandchildren
    #         for child in self.radial_children:
    #             child._cascade_radial_transform()

    # # Transform radial children AND their descendants
    # for i, child in enumerate(self._radial_list):
    #     if child and len(child._radial_list) > 0:
    #         # Calculate shift based on which position this child moved TO
    #         shift = self._calculate_radial_shift(i)

    #         # Recursively apply shift to entire subtree
    #         child._apply_radial_shift_recursive(shift)

    # def _calculate_radial_shift(self, position_index: int) -> int:
    #     """
    #     Calculate how much to shift a child's radial list based on:
    #     - Its position in parent's radial list
    #     - Direction of parent rotation
    #     """
    #     # For 4-fold symmetry (4 radial faces)
    #     # When parent rotates by -90°, child at position i needs shift:
    #     # Position 0 (new RIGHT): shift +1
    #     # Position 1 (new FRONT): shift +1
    #     # Position 2 (new LEFT):  shift -1
    #     # Position 3 (new BACK):  shift -1

    #     if len(self._radial_list) == 4:
    #         return 1 if position_index < 2 else -1

    #     # Add logic for 2-fold symmetry if needed
    #     return 0

    # def _apply_radial_shift_recursive(self, shift: int) -> None:
    #     """Recursively apply a radial shift to this node and all descendants."""
    #     if len(self._radial_list) == 0:
    #         return

    #     # Shift this node's radial children
    #     self._radial_list[:] = (
    #         self._radial_list[shift:] + self._radial_list[:shift]
    #     )

    #     # Recursively apply to all children (both axial and radial)
    #     for _, child in self.children:
    #         child._apply_radial_shift_recursive(shift)

    # if self._parent and self._parent._is_new_root:
    #     console.print(self.is_connected_to_axial_root(), self.id)
    # # if self.is_connected_to_axial_root():
    #     # this means this is either the top or the bottom
    #     # right, bottom, left, top
    # SHIFTS_PER_PLACE = {0: 1, 1: 1, 2: -1, 3: -1}

    # for i, child in enumerate(self._radial_list):
    #     if child and len(child._radial_list) > 0:  # Add check for non-empty list
    #         shift = SHIFTS_PER_PLACE.get(i, 0)
    #         child._radial_list[:] = child._radial_list[shift:] + child._radial_list[:shift]
    #         console.print(i, repr(child))
    #         console.print('changed child: ', child.node_id)

    # if self._parent and self._parent._is_new_root:
    #     for i, child in enumerate(self._radial_list):
    #         if child:
    #             # place 0 (rn right) -> shift with 1

    #             # place 1 (rn bottom) -> shift with -3

    #             # place 3 (rn left) -> shift with -1

    #             # place 4 (rn top) -> shift with 3

    #             # LEFT
    #             console.print(i, repr(child))
    #             child._radial_list[:] = child._radial_list[3:] + child._radial_list[:3]

    # # if a piece is the new root and the

    # # account for top/bottom for the root
    # if self._is_new_root:
    #     if self.TOP:
    #         console.print('found a top')
    #         # recurse everything to front?
    #         top_child = self.TOP
    #         for child in top_child.radial_children:
    #             console.print('rotated child ,', repr(child))
    #             child._radial_list[:] = child._radial_list[1:] + child._radial_list[:1]

    #     if self.BOTTOM:
    #         bottom_child = self.BOTTOM
    #         console.print('found a bottom')
    #         for child in bottom_child.radial_children:
    #             console.print('rotated child ', repr(child))
    #             child._radial_list[:] = child._radial_list[1:] + child._radial_list[:1]

    # for grand_child in child.radial_children:
    #     console.print('found the radial child of the axial child')
    #     grand_child._radial_list[:] = grand_child._radial_list[-1:] + grand_child._radial_list[:-1]

    def normalize_rotations(self) -> None:
        swap_amt = self.rotation // self._max_allowed_rotatations

        for _ in range(swap_amt):
            console.print(f"rotating {self.node_id}")
            self.rotate()

        for _, child in self.children:
            child.normalize_rotations()

    # def fix_new_root(self):
    #     for child in self.axial_children:
    #         console.print('found axial children:', repr(child))
    #         # for grand_child in child.radial_children:
    #         # child.LEFT:
    #         child.LEFT._radial_list[:] = child.LEFT._radial_list[-1:] + child.LEFT._radial_list[:-1]
    #         console.print('decreased the ggrand child rotaion')
    #         # child.RIGHT:
    #         child.RIGHT._radial_list[:] =  child.RIGHT._radial_list[-2:] + child.RIGHT._radial_list[:-2]

    def re_add_id(self, start_id: int = 0) -> int:
        self.node_id = start_id
        current_id = start_id + 1
        for child in self.children_nodes:
            current_id = child.re_add_id(current_id)
        return current_id

    def depth_first_collect(self) -> list[nx.DiGraph[Any]]:
        list_graph = []

        self._is_new_root = True
        # self.canonicalize_child_order()
        self.rotation = 0
        self.normalize_rotations()
        self.re_add_id()
        self._is_new_root = False

        list_graph.append(self.to_graph())

        for child in self.children_nodes:
            list_graph.extend(child.depth_first_collect())

        return list_graph

    # def normalize_rotations(self) -> None:
    #     swap_amt = self.rotation // self._max_allowed_rotatations

    #     for _ in range(swap_amt):
    #         console.print(f'rotating {self.node_id}')
    #         # Instead of self.rotate(), do normalization-specific transform
    #         self._normalize_one_step()

    #     for _, child in self.children:
    #         child.normalize_rotations()

    # def _normalize_one_step(self) -> None:
    #     """Normalize by one rotation step - THIS IS DIFFERENT FROM rotate()!"""
    #     self.rotation -= self._max_allowed_rotatations

    #     # Move to different face
    #     self._radial_list[:] = self._radial_list[-1:] + self._radial_list[:-1]

    #     # Axial children increment rotation
    #     for child in self.axial_children:
    #         child.rotation += self._max_allowed_rotatations

    #     # Radial children need FLIP transformation, not shift +2!
    #     for child in self.radial_children:
    #         child._cascade_flip_transform()

    # def _cascade_flip_transform(self) -> None:
    #     """Apply flip transformation when parent normalizes rotation."""
    #     if len(self._radial_list) >= 2:
    #         # Swap first two positions (RIGHT ↔ LEFT)
    #         self._radial_list[0], self._radial_list[1] = (
    #             self._radial_list[1], self._radial_list[0]
    #         )

    #     # Only recurse on radial children
    #     for child in self.radial_children:
    #         child._cascade_flip_transform()

    def canonicalize(self) -> CanonicalNode:
        # self.canonicalize_child_order()
        self.normalize_rotations()
        return self

---

In [ ]:
# from ariel_experiments.utils.initialize import generate_random_individual
from ariel_experiments.gui_vis.view_mujoco import view
from ariel_experiments.gui_vis.visualize_tree import (
    visualize_tree_from_graph,
)

console.is_jupyter = False  # makes the print output from rich more compact
%matplotlib inline


### Class usage demonstration

In [ ]:
# Create self!
diy_root = CanonicalNode.new_root()
diy_root.TOP = CanonicalNode(ModuleType.BRICK)  # you can add a module like this
diy_root.LEFT = CanonicalNode(
    "HINGE"
)  # or by simply typing (can also be lowercase)

# you can create modules first and stich them together
brick = CanonicalNode("brick")
brick2 = CanonicalNode("brick", 3)  # add rotation
brick.LEFT = brick2
hinge = CanonicalNode("hinge")
hinge.FRONT = brick

# you can also acces like a chain
diy_root.LEFT.FRONT = hinge

# when adding a chain a second time, a copy of the chain gets added:
diy_root.BOTTOM = hinge


hinge2 = CanonicalNode("hinge")
diy_root.BACK = hinge2

try:
    hinge2.LEFT = CanonicalNode("brick")  # Catches error if its not allowed
except ValueError as e:
    print(e)

# Canonical <-> Dict (_graph attribute in HPD)
diy_root_dict = diy_root.to_dict()
diy_root.from_dict(diy_root_dict)

# Canonical <-> Nx.Graph
diy_root_graph = diy_root.to_graph()
diy_root = CanonicalNode.from_graph(diy_root_graph)

# Print representations
console.print(diy_root_dict)
console.print(diy_root_graph)

console.print(diy_root)
console.print(repr(diy_root))

# Visualize!
visualize_tree_from_graph(diy_root_graph)
view(diy_root_graph)

---

### Test

test with only hinges

In [ ]:
root = CanonicalNode.new_root()

hinge_placement = ModuleFaces.TOP
angle = 2

connector = CanonicalNode("brick", angle)
# connector.TOP = CanonicalNode("brick")
connector[hinge_placement] = CanonicalNode("hinge")

brickthing = CanonicalNode("brick", 0)
# brickthing.TOP = CanonicalNode("brick")
brickthing[hinge_placement] = CanonicalNode("hinge")

connector.FRONT = brickthing
brickthing.FRONT = brickthing
# brickthing.FRONT.FRONT = brickthing

root.BACK = connector
root.LEFT = connector
root.TOP = connector
root.BOTTOM = connector
root.RIGHT = connector
root.FRONT = connector

view(root.to_graph(), with_viewer=False)

In [ ]:
print(CanonicalNode("brick")._radial_face_order)

top, 6 <br>
right, 4 <br>
bottom, 2 <br>
left,   0

for brick: <br>

a rotation normalisation (-2 rotation) means list_items should shift backward (place of bottom should become place of left)

In [ ]:
visualize_tree_from_graph(root.to_graph())

In [ ]:
root.canonicalize()

view(root.to_graph())
visualize_tree_from_graph(root.to_graph())

test with brick and hinges

In [ ]:
root = CanonicalNode.new_root()

hinge_placement = ModuleFaces.LEFT
angle = 6
connector = CanonicalNode("brick", angle)
connector.LEFT = CanonicalNode("hinge", 1)

connector.BOTTOM = CanonicalNode("brick")
connector.BOTTOM[hinge_placement] = CanonicalNode("hinge", 1)
connector.BOTTOM.TOP = CanonicalNode("hinge", 0)

brickthing = CanonicalNode("brick", angle)
brickthing.BOTTOM = CanonicalNode("brick")
brickthing.BOTTOM[hinge_placement] = CanonicalNode("hinge", 1)
brickthing.BOTTOM.TOP = CanonicalNode("hinge", 0)

connector.FRONT = brickthing
brickthing.FRONT = brickthing
# brickthing.FRONT.FRONT = brickthing

root.BACK = connector
root.LEFT = connector
root.TOP = connector
root.BOTTOM = connector
root.RIGHT = connector
root.FRONT = connector

view(root.to_graph(), with_viewer=False)
visualize_tree_from_graph(root.to_graph())

In [ ]:
root.canonicalize()

view(root.to_graph())
visualize_tree_from_graph(root.to_graph())

In [ ]:
print(root._radial_face_quat)

print(root)

In [ ]:
root = CanonicalNode.new_root()

brick1 = CanonicalNode("brick", 2)
# brick1.LEFT = CanonicalNode("hinge", 1)
brick1.TOP = CanonicalNode("hinge", 0)

brick2 = CanonicalNode("brick")
brick2.LEFT = CanonicalNode("hinge", 1)
brick2.TOP = CanonicalNode("hinge", 0)

brick1.BOTTOM = brick2

root.BACK = brick1
# root[placement].BOTTOM = brick # placement of the birck on brick

print(brick1._radial_face_order)

view(root.to_graph(), with_viewer=False)

In [ ]:
root.canonicalize()

view(root.to_graph())
# visualize_tree_from_graph(root.to_graph())

In [ ]:
root = CanonicalNode.new_root()

brick1 = CanonicalNode("brick", 2)
# brick1.LEFT = CanonicalNode("hinge", 1)
# brick1.TOP = CanonicalNode("hinge", 0)

brick2 = CanonicalNode("brick")
# brick2.LEFT = CanonicalNode("hinge", 1)
# brick2.TOP = CanonicalNode("hinge", 0)


brick3 = CanonicalNode("BRICK")
brick3.LEFT = CanonicalNode("hinge", 1)
brick3.TOP = CanonicalNode("Hinge")

brick2.RIGHT = brick3

brick1.RIGHT = brick2

root.BACK = brick1

root.TOP = CanonicalNode("Hinge")
# root[placement].BOTTOM = brick # placement of the birck on brick

print(brick1._radial_face_order)

view(root.to_graph(), with_viewer=False)

In [ ]:
root.canonicalize()

view(root.to_graph())
visualize_tree_from_graph(root.to_graph())

In [ ]:
root = CanonicalNode.new_root()

brick1 = CanonicalNode("brick", 0)
brick1.LEFT = CanonicalNode("hinge", 1)
brick1.TOP = CanonicalNode("hinge", 0)

# brick2 = CanonicalNode("brick")
# brick2.LEFT = CanonicalNode("hinge", 1)
# brick2.TOP = CanonicalNode("hinge", 0)


# brick3 = CanonicalNode("BRICK")
# brick3.LEFT = CanonicalNode("hinge", 1)
# brick3.TOP = CanonicalNode("Hinge")

# brick2.RIGHT = brick3

# brick1.RIGHT = brick2

root.BOTTOM = brick1

# root.TOP = CanonicalNode("Hinge")
# root[placement].BOTTOM = brick # placement of the birck on brick

# print(root._radial_face_order)

view(root.to_graph(), with_viewer=False)

In [ ]:
from ariel.body_phenotypes.robogen_lite.prebuilt_robots.gecko import gecko

gecko = gecko()

print(gecko)

In [ ]:
# Standard library
from pathlib import Path

import mujoco
import networkx as nx
import numpy as np

# Third-party libraries
from matplotlib import pyplot as plt
from mujoco import viewer
from PIL import Image
from rich.console import Console

from ariel.body_phenotypes.robogen_lite.config import (
    NUM_OF_FACES,
    NUM_OF_ROTATIONS,
    NUM_OF_TYPES_OF_MODULES,
)
from ariel.body_phenotypes.robogen_lite.constructor import (
    construct_mjspec_from_graph,
)
from ariel.body_phenotypes.robogen_lite.decoders.hi_prob_decoding import (
    HighProbabilityDecoder,
)
from ariel.body_phenotypes.robogen_lite.modules.core import CoreModule
from ariel.body_phenotypes.robogen_lite.prebuilt_robots.gecko import gecko

# Local libraries
from ariel.simulation.environments.simple_flat_world import SimpleFlatWorld
from ariel.utils.renderers import single_frame_renderer
from ariel_experiments.gui_vis.visualize_tree import VisualizationConfig

root: int = 0
title: str = ""
save_file: Path | str | None = None
config: VisualizationConfig | None = None
make_tree: bool = False
with_viewer: bool = True
robot = gecko()
viz_options = mujoco.MjvOption()  # visualization of various elements

# Visualization of the corresponding model or decoration element
viz_options.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True
viz_options.flags[mujoco.mjtVisFlag.mjVIS_ACTUATOR] = True
viz_options.flags[mujoco.mjtVisFlag.mjVIS_BODYBVH] = True

# MuJoCo basics
# world = TiltedFlatWorld()
world = SimpleFlatWorld(floor_size=(20, 20, 0.1))

for geom in world.spec.worldbody.geoms:
    if geom.name == "floor":
        geom.material = "custom_floor_material"
        break


world.spec.add_texture(
    name="custom_grid",
    type=mujoco.mjtTexture.mjTEXTURE_2D,
    builtin=mujoco.mjtBuiltin.mjBUILTIN_CHECKER,
    rgb1=[0.9, 0.9, 0.9],
    rgb2=[0.95, 0.95, 0.95],
    width=800,
    height=800,
)
world.spec.add_material(
    name="custom_floor_material",
    textures=["", "custom_grid"],
    texrepeat=[5, 5],
    texuniform=True,
    reflectance=0.05,
)

# Save the model to XML
if save_file:
    xml = robot.spec.to_xml()
    with (DATA / f"{save_file}.xml").open("w", encoding="utf-8") as f:
        f.write(xml)

# Make robot parts more transparant
for i in range(len(robot.spec.geoms)):
    robot.spec.geoms[i].rgba[-1] = 0.7

# Spawn the robot at the world
world.spawn(robot.spec)

# Compile the model
model = world.spec.compile()
data = mujoco.MjData(model)


# Number of actuators and DoFs
console.log(f"DoF (model.nv): {model.nv}, Actuators (model.nu): {model.nu}")

# Reset state and time of simulation
mujoco.mj_resetData(model, data)

# Render
img = single_frame_renderer(model, data, steps=10)

# View
if with_viewer:
    viewer.launch(model=model, data=data)

display(img)


In [ ]:
from ariel_experiments.utils.initialize import generate_random_individual
# from ariel_experiments.gui_vis import

In [ ]:
def remove_none(graph: DiGraph) -> DiGraph:
    """
    returns a graph without none types
    """

    none_list = []
    for node in graph.nodes():
        # finding all nones
        if graph.nodes(data=True)[node]["type"] == "NONE":
            none_list.append(node)

    # removing all nones
    for node in none_list:
        graph.remove_node(node)

    return graph

In [ ]:
individual = generate_random_individual()

visualize_tree_from_graph(individual)

individual_none_less = remove_none(individual)

visualize_tree_from_graph(individual_none_less)

canonical_individual = CanonicalNode.from_graph(individual)

visualize_tree_from_graph(canonical_individual.to_graph())

view(individual_none_less)

view(canonical_individual.to_graph())

In [ ]:
canonical_individual.canonicalize()

view(canonical_individual.to_graph())
visualize_tree_from_graph(canonical_individual.to_graph())

In [ ]:
diy_root = CanonicalNode.new_root("brick")

diy_root.rotation = 2

# diy_root.TOP  = CanonicalNode("brick")
# diy_root.BOTTOM = CanonicalNode("brick")
diy_root.LEFT = CanonicalNode("brick")
# diy_root.RIGHT = CanonicalNode("brick")
# diy_root.FRONT = CanonicalNode("brick")
# diy_root.BACK = CanonicalNode("brick")

console.print(diy_root._radial_face_quat)
console.print(diy_root._radial_face_order)

view(diy_root.to_graph(), with_viewer=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import quaternion as qnp
from mpl_toolkits.mplot3d import Axes3D

angle = 90

# Define original axes
axes = np.eye(3)  # X, Y, Z unit vectors

# Apply quaternion rotation
quat = qnp.from_euler_angles([
    np.deg2rad(180),
    -np.deg2rad(180 - angle),
    np.deg2rad(0),
])
rotation_matrix = qnp.as_rotation_matrix(quat)
rotated_axes = rotation_matrix @ axes

# Plot
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
colors = ["red", "green", "blue"]
labels = ["X", "Y", "Z"]

for i, (axis, color, label) in enumerate(zip(rotated_axes.T, colors, labels)):
    ax.quiver(
        0,
        0,
        0,
        axis[0],
        axis[1],
        axis[2],
        color=color,
        label=f"Rotated {label}",
    )

ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
ax.legend()
plt.show()


In [ ]:
shift = -1

---

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import quaternion as qnp
from mpl_toolkits.mplot3d import Axes3D

# Define shift (from original code)
shift = -1

# Create quaternion from Euler angles
original_quat = qnp.from_euler_angles([
    np.deg2rad(0),
    np.deg2rad(0),
    -np.deg2rad(90),
])

# Roll to MuJoCo format (for printing or MuJoCo use)
rolled_quat = np.round(
    np.roll(qnp.as_float_array(original_quat), shift=shift),
    decimals=3,
)

print("Rolled quaternion (MuJoCo format [x, y, z, w]):", rolled_quat)

# Define original axes (unit vectors)
axes = np.eye(3)  # X, Y, Z axes

# Get rotation matrix from the original quaternion (not the rolled array)
rotation_matrix = qnp.as_rotation_matrix(original_quat)
rotated_axes = rotation_matrix @ axes

# Plot
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
colors = ["red", "green", "blue"]
labels = ["X", "Y", "Z"]

for i, (axis, color, label) in enumerate(zip(rotated_axes.T, colors, labels)):
    ax.quiver(
        0,
        0,
        0,
        axis[0],
        axis[1],
        axis[2],
        color=color,
        label=f"Rotated {label}",
    )

ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
ax.legend()
plt.show()


---

In [ ]:
import numpy as np
import quaternion as qnp

# Parent rotates -90° around Y
q_parent_rot = qnp.from_euler_angles([0, np.deg2rad(-90), 0])

# Child on FRONT face
q_front_child = qnp.from_euler_angles([0, np.deg2rad(180), np.deg2rad(180)])
q_front_after = q_parent_rot * q_front_child

# Child on RIGHT face
q_right_child = qnp.from_euler_angles([
    np.deg2rad(90),
    np.deg2rad(90),
    -np.deg2rad(90),
])
q_right_after = q_parent_rot * q_right_child

# Convert back to euler to see the difference
euler_front = qnp.as_euler_angles(q_front_after)
euler_right = qnp.as_euler_angles(q_right_after)

print("Front child after parent rotation:", np.rad2deg(euler_front))
print("Right child after parent rotation:", np.rad2deg(euler_right))

In [ ]:
test_root = CanonicalNode.new_root()

brick1 = CanonicalNode("brick", 2)
brick2 = CanonicalNode("brick")

brick1.BOTTOM = brick2

hinge1 = CanonicalNode("hinge")
hinge2 = CanonicalNode("hinge", 1)

brick2.TOP = hinge1
brick2.LEFT = hinge2

test_root.LEFT = brick1

console.print()
console.print(test_root)

view(test_root.to_graph(), with_viewer=False)

In [ ]:
test_root.canonicalize()

console.print(test_root)

view(test_root.to_graph())

In [ ]:
import numpy as np
import quaternion as qnp

# Test case from your example:
# connector has rotation=2 (180°), attached to root.BACK
# After normalize: rotation should go to 0

# Child's rotation quaternion (from brick.rotate())
q_child_rot_180 = qnp.from_euler_angles([
    np.deg2rad(180),
    np.deg2rad(-90),  # -(180-180) = -0... wait, let me check
    0,
])

# Actually for rotation=2 (180°):
q_child_rot_180 = qnp.from_euler_angles([
    np.deg2rad(180),
    np.deg2rad(0),  # -(180-180) = 0
    0,
])

# Child on BACK face
q_back_face = qnp.from_euler_angles([
    np.deg2rad(0),
    np.deg2rad(0),
    np.deg2rad(0),
])

# Child's total orientation when on BACK with rotation=180
q_old = q_back_face * q_child_rot_180

# After normalization, rotation=0
q_child_rot_0 = qnp.from_euler_angles([
    np.deg2rad(180),
    np.deg2rad(-180),  # -(180-0)
    0,
])

# Which face should child be on now? Let's try FRONT
q_front_face = qnp.from_euler_angles([
    np.deg2rad(0),
    np.deg2rad(180),
    np.deg2rad(180),
])

q_new = q_front_face * q_child_rot_0

# Are they the same orientation?
print("Old orientation:", np.rad2deg(qnp.as_euler_angles(q_old)))
print("New orientation:", np.rad2deg(qnp.as_euler_angles(q_new)))

# What's the transformation in child's frame?
q_delta = q_new.conjugate() * q_old
R_delta = qnp.as_rotation_matrix(q_delta)

print("\nChild's radial faces transform as:")
for name, vec in [
    ("RIGHT", [1, 0, 0]),
    ("LEFT", [-1, 0, 0]),
    ("TOP", [0, 0, 1]),
    ("BOTTOM", [0, 0, -1]),
]:
    rotated = R_delta @ np.array(vec)
    print(f"  {name:6} -> {np.round(rotated, 2)}")

In [ ]:
import numpy as np
import quaternion as qnp

# For each rotation value, calculate what happens when we normalize by 1 step
# (moving to next radial face and decrementing rotation)


def test_normalization_step(start_rotation_value):
    """Test what happens to child's radial list when normalizing one step."""

    # Start rotation (in degrees)
    start_angle = start_rotation_value * 45  # Each step is 45°

    # Child's rotation quaternion at start
    q_start = qnp.from_euler_angles([
        np.deg2rad(180),
        -np.deg2rad(180 - start_angle),
        0,
    ])

    # After normalization: rotation decreases by 1, face shifts by 1
    end_angle = (start_rotation_value - 1) * 45
    q_end = qnp.from_euler_angles([
        np.deg2rad(180),
        -np.deg2rad(180 - end_angle),
        0,
    ])

    # Face quaternions (assuming start at position 0, end at position 1)
    # radial_face_order = [LEFT, BACK, RIGHT, FRONT] with ROTATION_DIRECTION = -1
    faces = [
        qnp.from_euler_angles([
            np.deg2rad(90),
            np.deg2rad(-90),
            np.deg2rad(-90),
        ]),  # LEFT
        qnp.from_euler_angles([
            np.deg2rad(0),
            np.deg2rad(0),
            np.deg2rad(0),
        ]),  # BACK
        qnp.from_euler_angles([
            np.deg2rad(90),
            np.deg2rad(90),
            np.deg2rad(-90),
        ]),  # RIGHT
        qnp.from_euler_angles([
            np.deg2rad(0),
            np.deg2rad(180),
            np.deg2rad(180),
        ]),  # FRONT
    ]

    # Old orientation: face[0] * rotation
    q_old = faces[0] * q_start

    # New orientation: face[1] * rotation
    q_new = faces[1] * q_end

    # Transformation in child's frame
    q_delta = q_new.conjugate() * q_old
    R_delta = qnp.as_rotation_matrix(q_delta)

    print(
        f"\nNormalizing from rotation={start_rotation_value} to rotation={start_rotation_value - 1}"
    )
    print(f"(moving from face position 0 to position 1)")
    print("Child's radial faces transform:")

    for name, vec in [
        ("RIGHT", [1, 0, 0]),
        ("LEFT", [-1, 0, 0]),
        ("TOP", [0, 0, 1]),
        ("BOTTOM", [0, 0, -1]),
    ]:
        rotated = R_delta @ np.array(vec)
        print(f"  {name:6} -> {np.round(rotated, 2)}")


# Test for rotation values that matter
for rot in [1, 2, 3, 4, 5, 6, 7]:
    test_normalization_step(rot)